# EEL891 - 2024.2 - Trabalho 2
### Regressão Multivariável - Estimar Preços de Imóveis
*Aluno:* Miguel Badany Cerne<br>*DRE:* 123370433<br>*Usuário Kaggle:* miguelbadanycerne<br>*ID Kaggle:* 20164449
<hr>

# 1. Baixar e Importar Bibliotecas

In [ ]:
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install scikit-learn
%pip install seaborn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 2. Exploração e Tratamento de Dados

In [11]:
df_treino = pd.read_csv('./conjunto_de_treinamento.csv')
df_treino.drop(['Id'], axis=1, inplace=True)
df_teste = pd.read_csv('./conjunto_de_teste.csv')
df_teste.drop(['Id'], axis=1, inplace=True)

In [12]:
df_treino.head(3).T

,0,1,2
tipo,Casa,Apartamento,Apartamento
bairro,Imbiribeira,Casa Amarela,Encruzilhada
tipo_vendedor,Imobiliaria,Imobiliaria,Imobiliaria
quartos,3,4,3
suites,3,4,1
vagas,5,2,0
area_util,223,157,53
area_extra,167,0,0
diferenciais,piscina e copa,piscina e churrasqueira,nenhum
churrasqueira,0,1,0


In [13]:
df_treino.columns

Index(['tipo', 'bairro', 'tipo_vendedor', 'quartos', 'suites', 'vagas',
       'area_util', 'area_extra', 'diferenciais', 'churrasqueira',
       'estacionamento', 'piscina', 'playground', 'quadra', 's_festas',
       's_jogos', 's_ginastica', 'sauna', 'vista_mar', 'preco'],
      dtype='object')

Notamos que as seguintes features não são numéricas:
- tipo
- bairro
- tipo_vendedor
- diferenciais<br>
Devemos tratar melhor cada uma dessas colunas posteriormente.

In [10]:
df_treino['diferenciais'].unique()

array(['piscina e copa', 'piscina e churrasqueira', 'nenhum',
       'playground', 'piscina e playground', 'sauna e playground',
       'piscina e sauna', 'piscina e quadra poliesportiva',
       'salao de festas', 'campo de futebol e playground',
       'piscina e estacionamento visitantes', 'copa e salao de festas',
       'esquina e salao de festas', 'piscina e salao de festas',
       'playground e sala de ginastica', 'copa e playground',
       'estacionamento visitantes e playground',
       'playground e salao de festas', 'churrasqueira e playground',
       'churrasqueira e copa', 'churrasqueira e salao de festas', 'copa',
       'esquina', 'piscina', 'playground e quadra poliesportiva',
       'estacionamento visitantes', 'piscina e esquina', 'copa e esquina',
       'estacionamento visitantes e salao de festas',
       'piscina e campo de futebol',
       'frente para o mar e salao de festas', 'churrasqueira',
       'churrasqueira e campo de futebol', 'piscina e frente para 

Para viabilizar o One-Hot Encoding para esta coluna, faremos um mapeamento dessas classes para diminuir o número total de classes.
1. Simplificamos as categorias, aglutinando todas combinações de mais de um elemento no nome do primeiro elemento, seguido de '++'
2. Anulamos os valores 'nenhum', 'churrasqueira', 'estacionamento'

In [23]:
valores_diferenciais = ['piscina e copa', 'piscina e churrasqueira', 'nenhum',
       'playground', 'piscina e playground', 'sauna e playground',
       'piscina e sauna', 'piscina e quadra poliesportiva',
       'salao de festas', 'campo de futebol e playground',
       'piscina e estacionamento visitantes', 'copa e salao de festas',
       'esquina e salao de festas', 'piscina e salao de festas',
       'playground e sala de ginastica', 'copa e playground',
       'estacionamento visitantes e playground',
       'playground e salao de festas', 'churrasqueira e playground',
       'churrasqueira e copa', 'churrasqueira e salao de festas', 'copa',
       'esquina', 'piscina', 'playground e quadra poliesportiva',
       'estacionamento visitantes', 'piscina e esquina', 'copa e esquina',
       'estacionamento visitantes e salao de festas',
       'piscina e campo de futebol',
       'frente para o mar e salao de festas', 'churrasqueira',
       'churrasqueira e campo de futebol', 'piscina e frente para o mar',
       'sala de ginastica e salao de festas', 'sauna e esquina',
       'sauna e salao de festas', 'frente para o mar', 'salao de jogos',
       'esquina e playground', 'frente para o mar e playground',
       'salao de festas e salao de jogos', 'sauna e campo de futebol',
       'esquina e quadra poliesportiva', 'piscina e sala de ginastica',
       'frente para o mar e copa', 'churrasqueira e sauna',
       'piscina e children care', 'frente para o mar e esquina',
       'copa e estacionamento visitantes', 'campo de futebol e esquina',
       'churrasqueira e estacionamento visitantes', 'sala de ginastica',
       'campo de futebol e estacionamento visitantes',
       'esquina e estacionamento visitantes',
       'frente para o mar e quadra poliesportiva',
       'piscina e quadra de squash',
       'quadra poliesportiva e salao de festas',
       'esquina e sala de ginastica', 'salao de festas e vestiario',
       'sauna e copa', 'piscina e hidromassagem',
       'churrasqueira e sala de ginastica', 'sauna e frente para o mar',
       'campo de futebol e salao de festas',
       'copa e quadra poliesportiva', 'churrasqueira e esquina',
       'sala de ginastica e salao de jogos',
       'churrasqueira e frente para o mar', 'piscina e salao de jogos',
       'sauna e sala de ginastica', 'sauna e quadra poliesportiva',
       'children care', 'copa e sala de ginastica',
       'playground e salao de jogos', 'quadra poliesportiva', 'vestiario',
       'children care e playground',
       'estacionamento visitantes e sala de ginastica',
       'frente para o mar e campo de futebol', 'campo de futebol e copa',
       'sauna', 'campo de futebol e quadra poliesportiva']

diferenciais = {valor: valor for valor in valores_diferenciais}

for valor in valores_diferenciais:
    if 'campo de futebol e' in valor[:18]:
        diferenciais[valor] = 'futebol++'
    if 'children care e' in valor[:15]:
        diferenciais[valor] = 'children care++'
    if 'churrasqueira e' in valor[:15]:
        diferenciais[valor] = 'churrasqueira++'
    if 'copa e' in valor[:6]:
        diferenciais[valor] = 'copa++'
    if 'esquina e' in valor[:9]:
        diferenciais[valor] = 'esquina++'
    if 'estacionamento visitantes e' in valor[:27]:
        diferenciais[valor] = 'estacionamento visita++'
    if 'frente para o mar e' in valor[:19]:
        diferenciais[valor] = 'mar++'
    if 'piscina e' in valor[:9]:
        diferenciais[valor] = 'piscina++'
    if 'playground e' in valor[:12]:
        diferenciais[valor] = 'playground++'
    if 'quadra poliesportiva e' in valor[:22]:
        diferenciais[valor] = 'quadra++'
    if 'sala de ginastica e' in valor[:19]:
        diferenciais[valor] = 'ginastica++'
    if 'salao de festas e' in valor[:17]:
        diferenciais[valor] = 'festas++'
    if 'salao de jogos e' in valor[:16]:
        diferenciais[valor] = 'festas++'
    if 'sauna e' in valor[:7]:
        diferenciais[valor] = 'sauna++'
    if valor == 'churrasqueira':
        diferenciais[valor] = np.nan
    if valor == 'estacionamento':
        diferenciais[valor] = np.nan
    if valor == 'piscina':
        diferenciais[valor] = np.nan
    if valor == 'playground':
        diferenciais[valor] = np.nan
    if valor == 'quadra poliesportiva':
        diferenciais[valor] = np.nan
    if valor == 'nenhum':
        diferenciais[valor] = np.nan
    

diferenciais

{'piscina e copa': 'piscina++',
 'piscina e churrasqueira': 'piscina++',
 'nenhum': nan,
 'playground': nan,
 'piscina e playground': 'piscina++',
 'sauna e playground': 'sauna++',
 'piscina e sauna': 'piscina++',
 'piscina e quadra poliesportiva': 'piscina++',
 'salao de festas': 'salao de festas',
 'campo de futebol e playground': 'futebol++',
 'piscina e estacionamento visitantes': 'piscina++',
 'copa e salao de festas': 'copa++',
 'esquina e salao de festas': 'esquina++',
 'piscina e salao de festas': 'piscina++',
 'playground e sala de ginastica': 'playground++',
 'copa e playground': 'copa++',
 'estacionamento visitantes e playground': 'estacionamento visita++',
 'playground e salao de festas': 'playground++',
 'churrasqueira e playground': 'churrasqueira++',
 'churrasqueira e copa': 'churrasqueira++',
 'churrasqueira e salao de festas': 'churrasqueira++',
 'copa': 'copa',
 'esquina': 'esquina',
 'piscina': nan,
 'playground e quadra poliesportiva': 'playground++',
 'estacionamen

In [24]:
df_treino = df_treino.replace(diferenciais)
df_teste = df_teste.replace(diferenciais)

In [ ]:
df_treino.head(10).T

,0,1,2
tipo,Casa,Apartamento,Apartamento
bairro,Imbiribeira,Casa Amarela,Encruzilhada
tipo_vendedor,Imobiliaria,Imobiliaria,Imobiliaria
quartos,3,4,3
suites,3,4,1
vagas,5,2,0
area_util,223,157,53
area_extra,167,0,0
diferenciais,piscina++,piscina++,NaN
churrasqueira,0,1,0
